ProjectTitle:Portugese Bank Marketing Project,
Project Id:PRCP-1000-PortugeseBank
Name :S K Yuvaraja,
Batch Id:31-MAY-CDS-25(Weekend 2:30 pm),
Project Team Id:PTID-CDS-JAN-26-3694, 
Email Id:yuvarajakannappan@gmail.com

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt

LOADING THE DATASET TO DATA FRAME

In [ ]:
df=pd.read_csv("Downloads/PRCP-1000-ProtugeseBank/Data/bank-additional/bank-additional-full.csv",sep=';')

In [ ]:
VIEWING THE LOADED DATA

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(data=df, x='y', y='duration')
plt.title('Call Duration Impact on Subscription')
plt.xlabel('Subscribed')
plt.ylabel('Call Duration (seconds)')
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(data=df, x='y', y='loan')
plt.title('loan vs y')
plt.xlabel('y')
plt.ylabel('loan')
plt.show()

In [ ]:
df.duplicated().sum()

In [ ]:
df[df.duplicated(keep=False)]

In [ ]:
df[df.duplicated()]


In [ ]:
df_clean = df.drop_duplicates()

In [ ]:
df_clean

DATA PREPROCESSING 

In [ ]:
categarical_cols=df_clean.select_dtypes(exclude='number')

In [ ]:
categarical_cols

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(data=categarical_cols, x='job', hue='y')
plt.xticks(rotation=45)
plt.title('Job vs Subscription')
plt.xlabel('Job')
plt.ylabel('Count')
plt.show()


In [ ]:
plt.figure(figsize=(5,4))
sns.countplot(data=categarical_cols, x='y')
plt.title('Class Distribution (Subscription)')
plt.xlabel('Subscribed')
plt.ylabel('Count')
plt.show()


In [ ]:
categarical_cols['y'].value_counts(normalize=True) * 100


In [ ]:
target_map = df_clean['y'].map({'no': 0, 'yes': 1})


In [ ]:
target_map.value_counts()

In [ ]:
numerical_cols=df_clean.select_dtypes(include='number')

In [ ]:
numerical_cols

In [ ]:
for col in numerical_cols:
    plt.figure(figsize=(6,5))
    sns.histplot(x=col,data=numerical_cols, kde=True)
    plt.title(f"Distribution of {col}")
    plt.show()

In [ ]:
numerical_cols.skew()

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(data=numerical_cols['duration'])
plt.xticks(rotation=50)
plt.yticks(rotation=50)
plt.title("Boxplot for All Columns")
plt.show()

In [ ]:
numerical_cols['duration'].skew()

In [ ]:
numerical_cols['duration_sqrt']=np.sqrt(numerical_cols['duration'])

In [ ]:
numerical_cols['duration_sqrt']

In [ ]:
plt.title('boxplot for Duration column after skew')
sns.boxplot(np.sqrt(numerical_cols['duration_sqrt']))
plt.show()

In [ ]:
numerical_cols['duration_sqrt'].skew()

In [ ]:
numerical_cols.shape

In [ ]:
numerical_cols

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(numerical_cols.corr(numeric_only=True), 
            annot=True, 
            cmap='coolwarm',
            fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
BINING NUMERICAL_COLS

In [ ]:
df_bl = numerical_cols.copy()

In [ ]:
age_bins = [0, 25, 35, 45, 55, 65, 100]
age_labels = [
    "Youth(<=25)", 
    "YoungAdult(26-35)", 
    "MidAge(36-45)", 
    "Senior(46-55)", 
    "PreRetire(56-65)", 
    "Retire(65+)"
]

df_bl["age_bin"] = pd.cut(df["age"], bins=age_bins, labels=age_labels)


In [ ]:
df_bl["age_bin"]

In [ ]:
if "duration_sqrt" not in df.columns:
    df["duration_sqrt"] = np.sqrt(df["duration"])

df_bl["duration_sqrt_bin"] = pd.cut(
    df["duration_sqrt"],
    bins=[0, 10, 13, 16, 19, np.inf],
    labels=[
        "VeryLow",
        "Low",
        "Medium",
        "High",
        "VeryHigh"
    ]
)



In [ ]:
df_bl["duration_sqrt_bin"]

In [ ]:
campaign_bins = [0, 1, 2, 3, 5, np.inf]
campaign_labels = [
    "SingleCall",
    "TwoCalls",
    "ThreeCalls",
    "4-5Calls",
    "HeavyContact(5+)"
]

df_bl["campaign_bin"] = pd.cut(
    df["campaign"],
    bins=campaign_bins,
    labels=campaign_labels
)


In [ ]:
df_bl["campaign_bin"]

In [ ]:
def pdays_bucket(x):
    if x == 999:
        return "NeverContacted"
    elif x <= 7:
        return "LastWeek"
    elif x <= 30:
        return "LastMonth"
    elif x <= 90:
        return "Last3Months"
    else:
        return "LongAgo"

df_bl["pdays_bin"] = df["pdays"].apply(pdays_bucket)


In [ ]:
df_bl["pdays_bin"]


In [ ]:
prev_bins = [-1, 0, 1, 2, 3, np.inf]
prev_labels = [
    "NoPrev",
    "OnePrev",
    "TwoPrev",
    "ThreePrev",
    "ManyPrev(4+)"
]

df_bl["previous_bin"] = pd.cut(
    df["previous"],
    bins=prev_bins,
    labels=prev_labels
)


In [ ]:
df_bl["previous_bin"]

In [ ]:
df_bl["emp_var_bin"] = pd.cut(
    df["emp.var.rate"],
    bins=[-5, -1, 0, 1, 5],
    labels=["StrongDown", "Down", "Up", "StrongUp"]
)


In [ ]:
df_bl["emp_var_bin"]

In [ ]:
df_bl["euribor_bin"] = pd.cut(
    df["euribor3m"],
    bins=[0, 1, 2, 3, 4, 6],
    labels=["VeryLow", "Low", "Medium", "High", "VeryHigh"]
)


In [ ]:
df_bl["euribor_bin"]

In [ ]:
df_bl["cons_conf_bin"] = pd.cut(
    df["cons.conf.idx"],
    bins=[-60, -50, -40, -30, -20, 0],
    labels=["VeryLow", "Low", "Neutral", "Good", "VeryGood"]
)


In [ ]:
df_bl["cons_conf_bin"]

In [ ]:
df_bl.filter(like="_bin").head(5)

In [ ]:
ENCODING THE BINED NUMERICAL_COLS

In [ ]:
df_bl_ohe = pd.get_dummies(
    df_bl,
    columns=df_bl.filter(like="_bin").columns,
    drop_first=True
)


In [ ]:
df_bl_ohe.drop(columns=['age','duration','campaign','pdays','previous','emp.var.rate','cons.price.idx','cons.conf.idx','euribor3m','nr.employed','duration_sqrt'], inplace=True)

In [ ]:
df_bl_ohe.dtypes.value_counts()

In [ ]:
df_bl_ohe

In [ ]:
ENCODING THE CATEGARICAL_COLS

In [ ]:
categarical_cols=["job", "marital", "contact", 
    "month", "day_of_week", "poutcome",'loan','housing',]
one_hot_encoded=pd.get_dummies(categarical_cols)
print(one_hot_encoded)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

education_order = [[
    'unknown',
    "illiterate",
    "basic.4y",
    "basic.6y",
    "basic.9y",
    "high.school",
    "professional.course",
    "university.degree"
]]

df[['education']] = OrdinalEncoder(categories=education_order).fit_transform(df[['education']])


In [ ]:
df[['education']]

In [ ]:
FEATURE SELCTION

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2, f_classif
X_cat = df_bl_ohe
y = target_map

chi_selector = SelectKBest(chi2, k=10)
chi_selector.fit(X_cat, y)
selected_cols = X_cat.columns[chi_selector.get_support()]

In [ ]:
selected_cols

In [ ]:
from sklearn.ensemble import RandomForestClassifier
X = df_bl_ohe
y = target_map

rf = RandomForestClassifier(random_state=42)
rf.fit(X, y)

importances = pd.Series(rf.feature_importances_, index=X.columns)
importances.sort_values(ascending=False).head(10)


In [ ]:
SLITING THE DATA

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
HYPER PERAMETER TUNNING

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators':[100,200],
    'max_depth':[5,10,None]
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1')
grid.fit(X_train, y_train)


In [ ]:
MODEL TRAINING

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)


In [ ]:
y_pred_lr

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

y_pred_dt = dt.predict(X_test)


In [ ]:
y_pred_dt

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)


In [ ]:
y_pred_rf

In [ ]:
MODEL EVALUATION

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

models = {
    "Logistic Regression": y_pred_lr,
    "Decision Tree": y_pred_dt,
    "Random Forest": y_pred_rf
}

for name, y_pred in models.items():
    print(f"\n{name}")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))


In [ ]:
TESTING THE BEST MODEL 

In [ ]:
best_model = rf

In [ ]:
customer_prob = best_model.predict_proba(df_bl_ohe)[:, 1]


In [ ]:
subscription_prob

In [ ]:
df_final = df_bl.copy()   # original dataset before encoding
df_final['customer_probability'] = customer_prob

In [ ]:
df_final

In [ ]:
df_final['customer_probability']

In [ ]:
df_final['target_segment'] = pd.cut(
    df_final['customer_probability'],
    bins=[0, 0.4, 0.7, 1.0],
    labels=['Low Priority', 'Medium Priority', 'High Priority']
)

In [ ]:
df_final['target_segment']

In [ ]:
high_value_customers = df_final[df_final['customer_probability'] >= 0.7]

high_value_customers[
    ['age', 'customer_probability']
].sort_values(by='customer_probability', ascending=False).head(10)

In [ ]:
df_final['marketing_action'] = np.where(
    df_final['customer_probability'] >= 0.7, 'Call Immediately',
    np.where(df_final['customer_probability'] >= 0.5, 'Send Email/SMS', 'No Campaign')
)


In [ ]:
df_final['marketing_action']

In [ ]:
df_final[['customer_probability', 'target_segment', 'marketing_action']].value_counts()

Suggestions to the Bank market team to make  customers  buy the product.
##Target high-probability customers
##Understand customer needs
##Improve call quality
##Personalize offers
##Optimize contact strategy
##Customize Offers Based on Customer Profile
##Explain benefits clearly


In [ ]:
MODEL CAMAPRISION

In [ ]:
model_comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree', 'Random Forest'],
    'Accuracy': [0.9060, 0.9019, 0.9036],
    'Precision (Class 1)': [0.68, 0.63, 0.64],
    'Recall (Class 1)': [0.32, 0.30, 0.33],
    'F1-score (Class 1)': [0.43, 0.41, 0.44]
})

model_comparison

In [ ]:
model_comparison.sort_values(by='Recall (Class 1)', ascending=False)


In [ ]:
model_comparison.sort_values(by='F1-score (Class 1)', ascending=False)


In [ ]:
model_comparison.set_index('Model')[[
    'Precision (Class 1)',
    'Recall (Class 1)',
    'F1-score (Class 1)'
]].plot(kind='bar', figsize=(10,5))

plt.title('Model Comparison on Positive Class (Subscription)')
plt.ylabel('Score')
plt.xticks(rotation=0)
plt.show()


Bank Marketing Prediction – Project report

The objective of this project is to build a predictive model that helps the bank marketing team identify customers who are most likely to subscribe to a product. By analyzing historical campaign data, the aim is to improve conversion rates while reducing unnecessary marketing efforts.

The dataset contains customer demographic details, financial attributes, past campaign information, and economic indicators. It includes both categorical and numerical variables and shows a clear class imbalance, with fewer customers subscribing to the product.

One of the initial challenges was the presence of duplicate records, which were identified and removed to maintain data quality. Another major challenge was handling mixed data types, for which categorical and numerical features were separated, and One-Hot Encoding was applied to categorical variables to make them suitable for machine learning models.

Correlation analysis revealed multicollinearity among economic indicators. This insight helped in choosing tree-based models that are less sensitive to correlated features. Some numerical variables were highly skewed, so binning techniques were applied to reduce the impact of extreme values and improve interpretability.

Feature selection was performed using Chi-Square tests and Random Forest feature importance to reduce dimensionality and avoid overfitting. Due to class imbalance, model evaluation focused on precision, recall, and F1-score instead of accuracy alone.

The data was split using stratified sampling to ensure fair evaluation, and hyperparameter tuning was carried out using GridSearchCV to improve model performance.

Logistic Regression, Decision Tree, and Random Forest models were trained and compared. Among them, Random Forest showed the best balance between precision and recall and was selected as the final model.
Random Forest
Accuracy: 0.9035939776590578
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      7308
           1       0.64      0.33      0.44       928

    accuracy                           0.90      8236
   macro avg       0.78      0.65      0.69      8236
weighted avg       0.89      0.90      0.89      823


Based on model predictions, customers were assigned subscription probability scores, enabling the marketing team to focus on high-probability customers, personalize offers, and optimize campaign strategies.